<a href="https://colab.research.google.com/github/vieduy/CS114.K21/blob/master/German_Traffic_Sign_Benchmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd drive/My\ Drive

/content/drive/My Drive


In [23]:
cd '..'

/content


In [0]:
!ls

'My Drive'


In [2]:
from config import dogs_vs_cats_config as config
print(config.NUM_TEST_IMAGES)

AttributeError: ignored

In [0]:
import matplotlib

# import the necessary packages
from scripts.preprocessing.imagetoarraypreprocessor import ImageToArrayPreprocessor
from scripts.preprocessing.simplepreprocessor import SimplePreprocessor
from scripts.preprocessing.patchpreprocessor import PatchPreprocessor
from scripts.preprocessing.meanpreprocessor import MeanPreprocessor
from scripts.callbacks.trainingmonitor import TrainingMonitor
from scripts.io.hdf5datasetgenerator import HDF5DatasetGenerator
from scripts.preprocessing.aspectawarepreprocessor import AspectAwarePreprocessor
from scripts.cnn.alexnet import AlexNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from scripts.io.simpledatasetloader import SimpleDatasetLoader
import json
import os
from imutils import paths
import random
from sklearn.model_selection import train_test_split
import numpy as np

In [0]:
aap = AspectAwarePreprocessor(64, 64)
iap = ImageToArrayPreprocessor()

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"duyhoangvien","key":"0952d645a4b4335830e2e94ecdd947d0"}'}

In [0]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw------- 1 root root 68 Jun  9 08:49 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# List available datasets.
!kaggle datasets list

In [15]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign --force

 98% 601M/612M [00:08<00:00, 89.2MB/s]
100% 612M/612M [00:08<00:00, 75.0MB/s]


In [24]:
!unzip gtsrb-german-traffic-sign.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/5/00005_00053_00011.png  
  inflating: train/5/00005_00053_00012.png  
  inflating: train/5/00005_00053_00013.png  
  inflating: train/5/00005_00053_00014.png  
  inflating: train/5/00005_00053_00015.png  
  inflating: train/5/00005_00053_00016.png  
  inflating: train/5/00005_00053_00017.png  
  inflating: train/5/00005_00053_00018.png  
  inflating: train/5/00005_00053_00019.png  
  inflating: train/5/00005_00053_00020.png  
  inflating: train/5/00005_00053_00021.png  
  inflating: train/5/00005_00053_00022.png  
  inflating: train/5/00005_00053_00023.png  
  inflating: train/5/00005_00053_00024.png  
  inflating: train/5/00005_00053_00025.png  
  inflating: train/5/00005_00053_00026.png  
  inflating: train/5/00005_00053_00027.png  
  inflating: train/5/00005_00053_00028.png  
  inflating: train/5/00005_00053_00029.png  
  inflating: train/5/00005_00054_00000.png  
  inflating: train/5/00005_00054_00001.png  
  in

In [29]:
imagePaths = list(paths.list_images('../../Train'))
random.shuffle(imagePaths)
length = len(imagePaths)
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=5000, length=length)
data = data.astype("float") / 255.0
print(labels)

[INFO] processed 5000/39209
[INFO] processed 10000/39209
[INFO] processed 15000/39209
[INFO] processed 20000/39209
[INFO] processed 25000/39209
[INFO] processed 30000/39209
[INFO] processed 35000/39209
[17 38 35 ... 12 17 36]


In [0]:
from sklearn.preprocessing import LabelBinarizer
import h5py

db = h5py.File('../../data.hdf5', 'r')
trainX = db["images"][:31368]
trainY = db["labels"][:31368]
testX = db["images"][31368:]
testY = db["labels"][31368:]

In [0]:
# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [6]:
# initialize the optimizer and model
from scripts.cnn.alexnet import AlexNet
print("[INFO] compiling model...")
opt = Adam(lr=1e-3)
model = AlexNet.build(width=64, height=64, depth=3,
  classes=43, reg=0.0002)
model.compile(loss="categorical_crossentropy", optimizer=opt,
  metrics=["accuracy"])

[INFO] compiling model...


In [7]:
print("[INFO] training network...")
model.fit(trainX, trainY, validation_data=(testX, testY),
  batch_size=64, epochs=40, verbose=1)

[INFO] training network...
Epoch 1/40
491/491 [==============================] - 10s 20ms/step - loss: 4.6418 - accuracy: 0.2560 - val_loss: 3.9107 - val_accuracy: 0.3471
Epoch 2/40
491/491 [==============================] - 9s 19ms/step - loss: 2.8817 - accuracy: 0.5556 - val_loss: 2.2722 - val_accuracy: 0.6813
Epoch 3/40
491/491 [==============================] - 9s 19ms/step - loss: 1.9531 - accuracy: 0.7590 - val_loss: 1.6577 - val_accuracy: 0.8211
Epoch 4/40
491/491 [==============================] - 9s 19ms/step - loss: 1.6164 - accuracy: 0.8248 - val_loss: 2.2375 - val_accuracy: 0.6614
Epoch 5/40
491/491 [==============================] - 9s 19ms/step - loss: 1.6180 - accuracy: 0.8416 - val_loss: 1.4460 - val_accuracy: 0.8676
Epoch 6/40
491/491 [==============================] - 9s 18ms/step - loss: 1.4478 - accuracy: 0.8769 - val_loss: 1.3439 - val_accuracy: 0.8982
Epoch 7/40
491/491 [==============================] - 9s 18ms/step - loss: 1.7614 - accuracy: 0.8696 - val_loss: 1

In [8]:
model.save('../../alexnet.model', overwrite=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../../alexnet.model/assets


In [9]:
from tensorflow.keras.models import load_model

print("[INFO] loading model...")
model = load_model('../../alexnet.model')

[INFO] loading model...


In [10]:
from os.path import basename, splitext

imagePaths = list((paths.list_images('../../Test')))
new_list = [splitext(basename(x))[0] for x in imagePaths]
fin_list = list(zip(imagePaths, new_list))
fin_list = [x[0] for x in sorted(fin_list,key=lambda x: int(x[1]))]
print(fin_list[0])
length = len(imagePaths)
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
data = sdl.load_test(fin_list, verbose=500, length=length)
data = data.astype("float") / 255.0

../../Test/00000.png
[INFO] processed 500/12630
[INFO] processed 1000/12630
[INFO] processed 1500/12630
[INFO] processed 2000/12630
[INFO] processed 2500/12630
[INFO] processed 3000/12630
[INFO] processed 3500/12630
[INFO] processed 4000/12630
[INFO] processed 4500/12630
[INFO] processed 5000/12630
[INFO] processed 5500/12630
[INFO] processed 6000/12630
[INFO] processed 6500/12630
[INFO] processed 7000/12630
[INFO] processed 7500/12630
[INFO] processed 8000/12630
[INFO] processed 8500/12630
[INFO] processed 9000/12630
[INFO] processed 9500/12630
[INFO] processed 10000/12630
[INFO] processed 10500/12630
[INFO] processed 11000/12630
[INFO] processed 11500/12630
[INFO] processed 12000/12630
[INFO] processed 12500/12630
12630


In [11]:
from sklearn.preprocessing import LabelBinarizer
import h5py

db = h5py.File('../../test.hdf5', 'r')
testImages = db["images"]
predictions = model.predict(testImages ,batch_size=64)
pred = predictions.argmax(axis=1)

In [12]:
import pandas as pd

test_df = pd.read_csv('Test.csv')
labels = test_df['ClassId'].values
print(labels[1000], pred[1000])

17 17


In [13]:
from sklearn.metrics import classification_report

print(classification_report(labels,
  predictions.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.97      0.58      0.73        60
           1       0.96      0.97      0.96       720
           2       0.94      0.92      0.93       750
           3       0.87      0.84      0.85       450
           4       0.97      0.92      0.94       660
           5       0.68      0.93      0.78       630
           6       0.79      0.75      0.77       150
           7       0.99      0.79      0.88       450
           8       0.96      0.89      0.93       450
           9       0.96      0.88      0.92       480
          10       0.94      0.92      0.93       660
          11       0.97      0.87      0.92       420
          12       0.99      0.94      0.96       690
          13       0.99      0.99      0.99       720
          14       0.99      1.00      0.99       270
          15       0.96      0.96      0.96       210
          16       1.00      1.00      1.00       150
          17       1.00    

In [14]:
 from sklearn.metrics import accuracy_score

 print(accuracy_score(labels, pred))

0.8974663499604117
